## Import dependent libraries

In [ ]:
import time
import os
import sys
import math
import random
from numpy import concatenate, array, asarray

from pandas import read_csv

from pandas import DataFrame
from pandas import concat

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

from matplotlib import pyplot
from IPython.display import SVG
import pydot
import graphviz

import tensorflow as tf

import keras
import keras.backend as K

from keras import optimizers
from keras.models import Model, model_from_json
from keras.layers import Input, Dense, Dropout, BatchNormalization
from keras.layers.recurrent import LSTM
from keras.utils import to_categorical
from keras.models import model_from_json
from keras.layers.core import Activation
from keras.callbacks import ReduceLROnPlateau, LambdaCallback
from keras.utils import plot_model
from sklearn.externals import joblib

from keras.utils.vis_utils import model_to_dot

from NN.ConvolutionalRNN import ConvolutionalRNN
from NN.jan_17_Model import SimpleModel
from Audio.MidiPlayer import MidiPlayer
from socketIO_client import SocketIO, LoggingNamespace


## Parameters

In [ ]:
# specify the number of lag hours
n_hours = 128
n_features = 2
n_train_hours = None
n_divisions = 4
batch_size = n_hours
epochs = 150
# player = MidiPlayer()
socket = SocketIO('localhost', 9876, LoggingNamespace)

## Load dataset

In [ ]:
dataset = read_csv('music_data.csv', header=0)
values = dataset.values
values = values.astype('float32')

## Specify columns to plot

In [ ]:
groups = [0, 1]
i = 1
# plot each column
pyplot.figure()
for group in groups:
    pyplot.subplot(len(groups), 1, i)
    pyplot.plot(values[:, group])
    pyplot.title(dataset.columns[group], y=0.5, loc='right')
    i += 1


pyplot.show()

## Convert series to supervised learning

In [ ]:
dataset = read_csv('music_data.csv', header=1)
n_train_hours = int(len(dataset) * 0.75)
values = dataset.values
values = values.astype('float32')

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    start = time.time()
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    end = time.time()
    print(end - start)
    return agg

## Scale Data

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(values)
scaled = scaler.transform(values)
print(scaled[20:21])


In [ ]:
# frame as supervised learning
reframed = series_to_supervised(scaled, n_hours, 1)

In [ ]:
values = reframed.values
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
# split into input and outputs
n_obs = n_hours * n_features

In [ ]:
train_X = train[:, :n_obs]
# print(train_X)

train_y_notes = train[:, -n_features]
print(train_y_notes[0:1])
train_y_volume = train[:, -n_features -1]

test_X = test[:, :n_obs]
# print(test_X)
test_y_notes = test[:, -n_features]
test_y_volume = test[:, -n_features -1]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
# print(train_X[0:10])

test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))
print(train_X.shape, train_y_notes.shape, test_X.shape, test_y_notes.shape)
print(test_X[0:1].shape)


In [ ]:
visible = Input(name='input', shape=(n_hours, n_features))
# conv_rnn = ConvolutionalRNN(visible, n_hours, n_features, n_divisions)
# simple = SimpleModel(visible, 128)

In [ ]:
# model = conv_rnn.model()
# model = simple.model()

hidden = LSTM(32, return_sequences=True)(visible)
hidden = LSTM(32, return_sequences=True)(hidden)
hidden = LSTM(32, return_sequences=True)(hidden)
hidden = LSTM(32, return_sequences=True)(hidden)
hidden = LSTM(32, return_sequences=True)(hidden)
hidden = LSTM(32, return_sequences=False)(hidden)
# hidden = BatchNormalization()(hidden)

output_notes = Dense(1, activation='sigmoid', name='output_notes')(hidden)
output_volume = Dense(1, activation='sigmoid', name='output_volume')(hidden)
# output_length = Dense(1, activation='sigmoid', name='output_length')(hidden_Z)

model = Model(inputs=[visible], outputs=[
                                         output_notes, 
                                         output_volume, 
#                                          output_length
                                        ])

In [ ]:
optimizer = optimizers.RMSprop(lr=0.0001)
model.compile(loss='mae', optimizer=optimizer)
# model.summary()

In [ ]:
plot_model(model, to_file='model.png')

In [ ]:
SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
test_cache = dataset.values[30:n_hours + 30].tolist()
test_data = test_cache.copy()
array_to_play = test_cache.copy()

In [ ]:
print(test_data)

In [ ]:
def midi_to_hertz(midi):
    if midi == 0:
        return 0
    g = 2**(1/12)
    return 440*g**(midi-69)

In [ ]:
def process_data(testx):
    testx = DataFrame(data = testx)
    testx = testx.values
    testx = testx.astype('float32')
    testx = scaler.transform(testx)
    return(testx)

In [ ]:
def on_epoch_end(epoch, logs):
    global test_cache
    global test_data
    global array_to_play
    if epoch > -5:
        start = time.time()
        print('----- Generating sound after: %d' % epoch)
        for i in range(400):
            if i == 0:
                to_add = [100, 80]
                array_to_play.append(to_add)
            if i == 100 or i == 500:  
                rand = random.randint(10, 15)
                for i in range(rand):
                    to_add = [0, 0]
                    array_to_play.append(to_add)
                    test_data.append(to_add)
                for i in range(rand):
                    test_data.pop(0)

            if i == 300:    
                for i in range(15):
                    rand = random.randint(45, 65)
                    to_add = [rand, 45]
                    array_to_play.append(to_add)
                    test_data.append(to_add)
                for i in range(15):
                    test_data.pop(0)

            data = process_data(test_data)
            data = data.reshape(1,n_hours,2)
            prediction = model.predict(data)
            prediction = asarray(prediction).ravel().reshape(-1,n_features)
            prediction = scaler.inverse_transform(prediction)
            prediction = prediction.astype('int')
            prediction = prediction[0].tolist()
            test_data.append(prediction)
            test_data.pop(0)
            array_to_play.append(prediction)
        end = time.time()
        print('time:', end - start)
        print('len array_to_play', len(array_to_play))
        for value in array_to_play:
            freq = midi_to_hertz(value[0])
            socket.emit('freq_change', {'freq': freq, 'vol': 100})
            time.sleep(.03)
        socket.emit('freq_change', {'freq': 0, 'vol': 100})
        test_data = test_cache.copy()
        array_to_play = test_cache.copy()

    
play_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:
history = model.fit(
            {'input': train_X},
            {
                'output_notes': train_y_notes, 
                'output_volume': train_y_volume, 
            },
#             validation_data=({'input': test_X},
#                              {
#                                  'output_notes': test_y_notes, 
#                                  'output_volume': test_y_volume, 
#                              }), 
            verbose=1,
            shuffle=False,
            epochs=epochs, 
            batch_size=int(batch_size),
            callbacks=[play_callback]
)

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

In [ ]:
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
joblib.dump(scaler, 'scaler.pkl')
print('Saved scaler to disk.')

In [ ]:
test_data = test_cache.copy()
array_to_play = test_cache.copy()